In [67]:
import math

def modular_pow(base, exponent, modulus):
    '''
    Calculates (base^exponent) % modulus efficiently.
    This function uses the right-to-left binary method.
    '''
    result = 1
    base %= modulus
    while exponent > 0:
        # If exponent is odd, multiply base with the result
        if exponent % 2 == 1:
            result = (result * base) % modulus
        # Exponent must be even now
        exponent = exponent >> 1  # Equivalent to exponent //= 2
        base = (base * base) % modulus
    return result

def jacobi_symbol(a, n):
    '''
    Calculates the Jacobi symbol (a/n) using a standard iterative algorithm.
    n must be a positive, odd integer.
    '''
    if n <= 0 or n % 2 == 0:
        raise ValueError("Jacobi symbol is only defined for positive odd n.")
    t = 1
    a %= n
    t = 1
    while a != 0:
        # Rule: Factor out all powers of 2 from 'a'
        while a % 2 == 0:
            a //= 2
            n_mod_8 = n % 8
            # Rule (2/n) = (-1)^((n^2-1)/8)
            if n_mod_8 in (3, 5):
                t = -t  # Flip the sign
        # At this point, 'a' is guaranteed to be odd.
        # Rule: Quadratic Reciprocity
        # Swap 'a' and 'n'
        a, n = n, a
        # Apply sign change if a and n were both congruent to 3 (mod 4)
        if a % 4 == 3 and n % 4 == 3:
            t = -t
        # Rule: (a/n) = (a mod n / n) for the next iteration
        a %= n
    if n == 1:
        return t
    else:  # This case occurs if the original gcd(a, n) was > 1
        return 0

In [68]:
def is_prime(n):
    '''
    A number n is prime if it is not divisible by any integer t with 1 < t <= sqrt(n).
    '''
    if n <= 1:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def fermat_test(N, a):
    '''
    Performs the Fermat primality test for a number N with base a.
    '''
    if modular_pow(a, N - 1, N) != 1:
        return False
    return True


def euler_test(N, a):
    '''
    Performs the Euler primality test for an odd number N with base a.
    The test passes if a^((N-1)/2) is congruent to the Jacobi symbol (a/N) mod N.
    '''
    if N <= 1 or N % 2 == 0:
        return False
    jacobi = jacobi_symbol(a, N)
    if jacobi == 0:
        return False
    lhs = modular_pow(a, (N - 1) // 2, N)
    rhs = N - 1 if jacobi == -1 else 1

    return lhs == rhs

def miller_rabin_test(N, a):
    '''
    Performs the strong primality test (Miller-Rabin) for a number N with base a.
    N must be a positive, odd integer greater than 2.
    '''
    if N <= 1 or N % 2 == 0:
        return False
    if a <= 1 or a >= N:
        raise ValueError("Base 'a' must be between 1 and N-1.")
    s = N - 1
    r = 0
    while s % 2 == 0:
        s //= 2
        r += 1
    x = modular_pow(a, s, N)
    if x == 1:
        return True
    for _ in range(r):
        if x == N - 1:
            return True
        x = modular_pow(x, 2, N)
    return False

def run_test_on_intervals(start, end, max_a, test):
    '''
    Runs the selected primality test on an interval for bases up to base a
    '''
    pseudoprimes = []
    prime_numbers = []
    for N in range(start, end + 1):
        if N <= 1:
            continue
        is_probable_prime = True
        for a in range(2, max_a + 1):
            if not test(N, a):
                is_probable_prime = False
                break
        if is_probable_prime:
            if not is_prime(N):
                pseudoprimes.append(N)
            else:
                prime_numbers.append(N)
    return pseudoprimes, prime_numbers

In [69]:
# Get the number of base 2 pseudoprimes for each test.
k_powers = range(5, 10)
tests = [fermat_test, euler_test, miller_rabin_test]
print("Base 2 pseudoprimes")
print("-" * 50)
print(f"{'k':<4} | {'Primes':<8} | {'Fermat':<8} | {'Euler':<8} | {'Strong':<8}")
print("-" * 50)
for k in k_powers:
    start = 10**k
    end = 10**k + 10**5
    base = 2
    row = []
    for test in tests:
        pseudoprimes, prime_numbers = run_test_on_intervals(start, end, base, test)
        if test == fermat_test:
            row.append(len(run_test_on_intervals(start, end, base, test)[1]))
        row.append(len(run_test_on_intervals(start, end, base, test)[0]))
    print(f"{k:<4} | {row[0]:<8} | {row[1]:<8} | {row[2]:<8} | {row[3]:<8}")

Base 2 pseudoprimes
--------------------------------------------------
k    | Primes   | Fermat   | Euler    | Strong  
--------------------------------------------------
5    | 8392     | 28       | 13       | 3       
6    | 7216     | 16       | 9        | 4       
7    | 6241     | 6        | 4        | 0       
8    | 5411     | 1        | 0        | 0       
9    | 4832     | 0        | 0        | 0       


In [71]:
# Get the number of base 2 and 3 pseudoprimes for each test.
k_powers = range(5, 10)
tests = [fermat_test, euler_test, miller_rabin_test]
print("Base 2 and 3 pseudoprimes")
print("-" * 50)
print(f"{'k':<4} | {'Primes':<8} | {'Fermat':<8} | {'Euler':<8} | {'Strong':<8}")
print("-" * 50)
for k in k_powers:
    start = 10**k
    end = 10**k + 10**5
    base = 3
    row = []
    for test in tests:
        pseudoprimes, prime_numbers = run_test_on_intervals(start, end, base, test)
        if test == fermat_test:
            row.append(len(run_test_on_intervals(start, end, base, test)[1]))
        row.append(len(run_test_on_intervals(start, end, base, test)[0]))
    print(f"{k:<4} | {row[0]:<8} | {row[1]:<8} | {row[2]:<8} | {row[3]:<8}")

Base 2 and 3 pseudoprimes
--------------------------------------------------
k    | Primes   | Fermat   | Euler    | Strong  
--------------------------------------------------
5    | 8392     | 9        | 3        | 0       
6    | 7216     | 4        | 2        | 0       
7    | 6241     | 2        | 2        | 0       
8    | 5411     | 1        | 0        | 0       
9    | 4832     | 0        | 0        | 0       


There is a hierarchy of strength based upon the number of pseudoprimes detected: Fermat > Euler > Strong. For any given base, the number of composite numbers that can falsely passs the test decreases with each successive refinement. This is because the tests are built upon each other; a number that passes the strong test for base $a$ is guaranteed to pass the Euler test for $a$, and a number that passes the Euler test is guaranteed to pass the Fermat test.

In all intervals, the number of pseudoprimes for any test is exceptionally small compared to the number of actual primes. This demonstrates that all of the tests are quite effective at filtering out the vast majority of composite numbers. Simply by requiring a number to pass a test for two small bases, the number of pseudoprimes drops dramatically.

The strong test is demonstrably the most reliable. Even for a single base, it allows the fewest pseudoprimes to pass. When combined with a second base, it becomes a deterministic primality test for the entire range of numbers examined. Furthermore, the likelihood of a pseudoprime drops as $k$ increases, likely due to the increasing sparsity of prime numbers.

In [89]:
import random
import time

def is_prime_efficient(n):
    '''
    An improvement using a mix of algorithms: Combines pre-screening with a
    deterministic Miller-Rabin test for the range up to 10^10.
    '''
    if n < 2:
        return False

    # Set of bases that guarantee primality for n up to our range and beyond
    deterministic_bases = [2, 3]

    # Pre-screening and direct checks
    if n in deterministic_bases:
        return True
    for base in deterministic_bases:
        if n % base == 0:
            return False

    # Deterministic Miller-Rabin test for numbers that pass pre-screening
    for a in deterministic_bases:
        if not miller_rabin_test(n, a):
            return False
    return True

num_samples = 10000
test_range_start = 3
test_range_end = 10**10

print(f"Generating {num_samples} random numbers between {test_range_start} and {test_range_end}...")
random_numbers = [random.randint(test_range_start, test_range_end) for _ in range(num_samples)]

# Time the trial division algorithm
start_time_trial = time.perf_counter()
for num in random_numbers:
    is_prime_efficient(num)
end_time_trial = time.perf_counter()
trial_duration = end_time_trial - start_time_trial
print(f"Trial Division took: {trial_duration:.4f} seconds.")

# Time the efficient algorithm
start_time_efficient = time.perf_counter()
for num in random_numbers:
    is_prime_efficient(num)
end_time_efficient = time.perf_counter()
efficient_duration = end_time_efficient - start_time_efficient
print(f"Efficient Algorithm took: {efficient_duration:.4f} seconds.")

Generating 10000 random numbers between 3 and 10000000000...
Trial Division took: 0.0539 seconds.
Efficient Algorithm took: 0.0492 seconds.


We want to find the probability that a number $N$ is composite given that it has passed $t$ rounds of the strong test. Define the events:
*   $C$: The event that $N$ is composite.
*   $P$: The event that $N$ is prime, the complement of $C$.
*   $T_t$: The event that $N$ passes $t$ rounds of the strong test with randomly chosen bases.

We want to calculate the conditional probability $\Pr(C \vert T_t)$ where we apply Bayes' theorem
\begin{equation}
    \Pr(C | T_t) = \frac{\Pr(T_t \vert C) \Pr(C)}{\Pr(T_t)}.
\end{equation}
The denominator can be expanded using the law of total probability,
\begin{equation}
    \Pr(T_t) = \Pr(T_t \vert C) \Pr(C) + \Pr(T_t \vert P) \Pr(P)
\end{equation}
The prior probabilities $\Pr(C)$ and $\Pr(P)$ are that a randomly chosen $k$-bit odd integer is either composite or prime. We can estimate this using the prime number theorem, which states that the density of primes around a number $x$ is approximately $1 / \log(x)$. For a k-bit number, $N$ is approximately $2^k$, so $\log(N)$ is approximately $\log(2^k) = k\log(2)$. Hence
\begin{equation}
    \Pr(P) \approx \frac{1}{k\log(2)}, \quad \Pr(C) ≈ 1 - \frac{1}{k\log(2)}.
\end{equation}

*   Test reliability on primes $\Pr(T_t \vert P)$.
This is the probability that a number passes $t$ tests given that it is prime. By the theorem on which the Miller-Rabin test is based, a prime number will always pass the test, regardless of the base chosen. Therefore,
\begin{equation}
    \Pr(T_t \vert P) = 1
\end{equation}
*   Test reliability on composites $\Pr(T_t \vert C)$.
This is the probability that a number passes $t$ tests given that it is composite. For any composite number $N$, the number of bases for which $N$ passes the test is at most $(N-1)/4$. Therefore, $\Pr(T_1 \vert C) \leq 1/4$.
Since the $t$ bases are chosen independently,
\begin{equation}
    \Pr(T_t \vert C) \leq \frac{1}{4^t}
\end{equation}
In practice, for most composite numbers, the fraction of 'liar' bases is far smaller than $1/4$, so this is a very conservative estimate.

Substitute these components back into the Bayes' formula to obtain
\begin{equation}
    \Pr(C \vert T_t) \approx \frac{(1/4)^t (1 - 1/(k\log{2}))} {(1/4)^t (1 - 1/(k\log{2})) + (1/(k\log{2})) }.
\end{equation}
The term $(1 - 1/(k\log{2}))$ is very close to 1. The numerator is dominated by the very small $(1/4)^t$. The denominator is a sum of that very small number and a much larger number $(1/(k\log{2}))$. Therefore, the entire fraction is extremely small.

As k and t increase, this probability becomes so infinitesimally small that it is far lower than the probability of a hardware error during the computation.